# Customize vanilla MONAILabel `radiology` app
In this document, we will review how to install MONAILabel, install the default (vanilla) `radiology` app, and customize the segmentation app for our needs.

## Structure of vanilla MONAILabel `radiology` app
**Please refer to the included `dicom2deployment-initial-setup.ipynb` notebook for instructions on installing the MONAILabel radiology app.**

The folder structure of the vanilla MONAILabel app will have the following directory tree. We start by editing the default segmentation.py config file shown below.

```
apps
└── radiology
    └── bin
    └── lib
        └── configs
            ├── segmentation.py
            ...
        └── infers
            ...
        └── trainers
            ...
    └── model
        ...
    ...
```

## Customization
For this project, we want to customize the segmentation app to do a multi-label segmentation of the eyeballs (globes) from head CT images. The following subsections briefly introduce the customization of the supporting scripts for the radiology segmentation app. 

### Copying the segmentation config
First we will copy `./apps/radiology/lib/configs/segmentation.py` to `./apps/radiology/lib/configs/segmentation_globes.py` so that we don't have to change the original file.

In [3]:
import shutil

_ = shutil.copyfile('./apps/radiology/lib/configs/segmentation.py', './apps/radiology/lib/configs/segmentation_globes.py')

### Customize labels

Now that we have a new config file to edit, we will change the expected labels. Open the newly created `./apps/radiology/lib/configs/segmentation.py` file in your favorite python editor. The vanilla segmentation config includes the following labels:

```python
# Labels
self.labels = {
    "spleen": 1,
    "kidney_right": 2,
    "kidney_left": 3,
    "gallbladder": 4,
    "liver": 5,
    "stomach": 6,
    "aorta": 7,
    "inferior_vena_cava": 8,
    "portal_vein_and_splenic_vein": 9,
    "pancreas": 10,
    "adrenal_gland_right": 11,
    "adrenal_gland_left": 12,
    "lung_upper_lobe_left": 13,
    "lung_lower_lobe_left": 14,
    "lung_upper_lobe_right": 15,
    "lung_middle_lobe_right": 16,
    "lung_lower_lobe_right": 17,
    "esophagus": 42,
    "trachea": 43,
    "heart_myocardium": 44,
    "heart_atrium_left": 45,
    "heart_ventricle_left": 46,
    "heart_atrium_right": 47,
    "heart_ventricle_right": 48,
    "pulmonary_artery": 49,
}
```

We will change this to right and left globes for our segmentation config:
```python
# Labels
self.labels = {
            "right_globe": 1,
            "left_globe": 2,
}
```

### Disable pretrained models
MONAILabel has several pretrained models that may be useful for your projects. However, there is currently no pretrained CT model that has exactly 2 labels. For this reason, we will disable the pretrained model download and start from scratch. To accomplish this all we need to do is modify the following line of code:
```python
# Download PreTrained Model
if strtobool(self.conf.get("use_pretrained_model", "true")):
            url = f"{self.conf.get('pretrained_path', self.PRE_TRAINED_PATH)}"
            url = f"{url}/radiology_segmentation_segresnet_multilabel.pt"
            download_file(url, self.path[0])
```
and change it to (only change is `true` to `false` in first line:
```python
# Download PreTrained Model
if strtobool(self.conf.get("use_pretrained_model", "false")):
            url = f"{self.conf.get('pretrained_path', self.PRE_TRAINED_PATH)}"
            url = f"{url}/radiology_segmentation_segresnet_multilabel.pt"
            download_file(url, self.path[0])
```

### Add active learning modules
For the last step of our customization, we will add a few codeblocks that enable calculation of epistemic loss. This will allow us to use active learning to accelerate data labeling in MONAILabel. First we need to add some additional imports at the top of the script:

```python
from monailabel.interfaces.tasks.scoring import ScoringMethod
from monailabel.interfaces.tasks.strategy import Strategy
from monailabel.tasks.activelearning.epistemic import Epistemic
from monailabel.tasks.scoring.dice import Dice
from monailabel.tasks.scoring.epistemic import EpistemicScoring
from monailabel.tasks.scoring.sum import Sum
```

Next, we will add the following lines to the end of the `init` method of the `Segmentation` class.
```python
# for epistemic uncertainty
self.epistemic_enabled = strtobool(conf.get("epistemic_enabled", "false"))
self.epistemic_samples = int(conf.get("epistemic_samples", "5"))
logger.info(f"EPISTEMIC Enabled: {self.epistemic_enabled}; Samples: {self.epistemic_samples}")
```
Next we will add the following two additional methods at the end of the `Segmentation` class:

```python
def strategy(self) -> Union[None, Strategy, Dict[str, Strategy]]:
    strategies: Dict[str, Strategy] = {}
    if self.epistemic_enabled:
        strategies[f"{self.name}_epistemic"] = Epistemic()
    return strategies

def scoring_method(self) -> Union[None, ScoringMethod, Dict[str, ScoringMethod]]:
    methods: Dict[str, ScoringMethod] = {
        "dice": Dice(),
        "sum": Sum(),
    }

    if self.epistemic_enabled:
        methods[f"{self.name}_epistemic"] = EpistemicScoring(
            model=self.path,
            network=self.network,
            transforms=lib.infers.Segmentation(None).pre_transforms(),
            num_samples=self.epistemic_samples,
        )
    return methods
```

# Finished!
Congrats! Now we have successfully modified the MONAILabel radiology segmentation app for our own project. The final step is deploying your segmentation app to your collaborators.